Build the model 30 times and calculate the accuracy.

In [1]:
%config Completer.use_jedi=False
import tensorflow as tf
from tensorflow import keras
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense,Dropout,BatchNormalization,Flatten,Conv1D,MaxPooling1D,Input,LSTM
from tensorflow.keras.layers import concatenate
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
from tensorflow.keras.backend import argmax
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import RobustScaler, StandardScaler
from scipy.stats import mode
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import os
from os import path

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# from http://www.johnvinyard.com/blog/?p=268

import numpy as np
from numpy.lib.stride_tricks import as_strided as ast

def norm_shape(shape):
    '''
    Normalize numpy array shapes so they're always expressed as a tuple,
    even for one-dimensional shapes.
    Parameters
        shape - an int, or a tuple of ints
    Returns
        a shape tuple
    '''
    try:
        i = int(shape)
        return (i,)
    except TypeError:
        # shape was not a number
        pass

    try:
        t = tuple(shape)
        return t
    except TypeError:
        # shape was not iterable
        pass

    raise TypeError('shape must be an int, or a tuple of ints')

def sliding_window(a,ws,ss = None,flatten = True):
    '''
    Return a sliding window over a in any number of dimensions
    Parameters:
        a  - an n-dimensional numpy array
        ws - an int (a is 1D) or tuple (a is 2D or greater) representing the size
             of each dimension of the window
        ss - an int (a is 1D) or tuple (a is 2D or greater) representing the
             amount to slide the window in each dimension. If not specified, it
             defaults to ws.
        flatten - if True, all slices are flattened, otherwise, there is an
                  extra dimension for each dimension of the input.
    Returns
        an array containing each n-dimensional window from a
    '''

    if None is ss:
        # ss was not provided. the windows will not overlap in any direction.
        ss = ws
    ws = norm_shape(ws)
    ss = norm_shape(ss)

    # convert ws, ss, and a.shape to numpy arrays so that we can do math in every
    # dimension at once.
    ws = np.array(ws)
    ss = np.array(ss)
    shape = np.array(a.shape)


    # ensure that ws, ss, and a.shape all have the same number of dimensions
    ls = [len(shape),len(ws),len(ss)]
    if 1 != len(set(ls)):
        raise ValueError(\
        'a.shape, ws and ss must all have the same length. They were %s' % str(ls))

    # ensure that ws is smaller than a in every dimension
    if np.any(ws > shape):
        raise ValueError(\
        'ws cannot be larger than a in any dimension.\
 a.shape was %s and ws was %s' % (str(a.shape),str(ws)))

    # how many slices will there be in each dimension?
    newshape = norm_shape(((shape - ws) // ss) + 1)
    # the shape of the strided array will be the number of slices in each dimension
    # plus the shape of the window (tuple addition)
    newshape += norm_shape(ws)
    # the strides tuple will be the array's strides multiplied by step size, plus
    # the array's strides (tuple addition)
    newstrides = norm_shape(np.array(a.strides) * ss) + a.strides
    strided = ast(a,shape = newshape,strides = newstrides)
    if not flatten:
        return strided

    # Collapse strided so that it has one more dimension than the window.  I.e.,
    # the new array is a flat list of slices.
    meat = len(ws) if ws.shape else 0
    firstdim = (np.product(newshape[:-meat]),) if ws.shape else ()
    dim = firstdim + (newshape[-meat:])
    # remove any dimensions with size 1
    #dim = filter(lambda i : i != 1,dim) #sytan modified here
    return strided.reshape(dim)

In [3]:
# Hardcoded number of sensor channels employed in the gyro X challenge - X,Y,Z
SENSOR_CHANNELS = 6
# Hardcoded number of classes in the gesture recognition problem
NUM_CLASSES = 12

# sampling rate: 50Hz, 100 * 1/50 = 2 seconds
# Hardcoded length of the sliding window mechanism employed to segment the data
SLIDING_WINDOW_LENGTH = 128
# Hardcoded step of the sliding window mechanism employed to segment the data (50% overlapping)
SLIDING_WINDOW_STEP = 64

# current path
cwd = os.getcwd()

In [4]:
def debug_print_count(data, string='NA'):
    unique, counts = np.unique(data, return_counts=True)
    print(string)
    print(dict(zip(unique, counts)))

In [5]:
# load a single file as a numpy array
def load_file(filepath):
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

In [6]:
def load_dataset_group(dataset_type):
    if (dataset_type == 'train'):
        #path = 'https://raw.githubusercontent.com/sytan6419/hapt_test/main/traindata.txt'
        path = '../traindata.txt'
    elif (dataset_type == 'test'):
        #path = 'https://raw.githubusercontent.com/sytan6419/hapt_test/main/testdata.txt'
        path = '../testdata.txt'

    data = load_file(path)
    return data[:,0:SENSOR_CHANNELS], data[:,-1]

In [7]:
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train')
    print("Train set:" + str(trainX.shape) + str(trainy.shape))
    # load all test
    testX, testy = load_dataset_group('test')
    print("Test set:" + str(testX.shape) + str(testy.shape))

    return trainX, trainy, testX, testy

In [8]:
'''
@data_y: take the mode of the array as the activity label.
'''
def har_sliding_window(data_x, data_y, ws, ss):
    data_x = sliding_window(data_x,(ws,data_x.shape[1]),(ss,1))
    data_y = mode(sliding_window(data_y,ws,ss), axis=1)[0] 
    #data_y = np.asarray([[i[-1]] for i in sliding_window(data_y,ws,ss)])
    
    return data_x.astype(np.float32), data_y.reshape(len(data_y)).astype(np.uint8)

In [9]:
# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [10]:
# Plot the data
def plot_data(original_data, scaled_data=None):
    # plot both together to compare
    #fig, ax=plt.subplots(1,2)
    fig, ax=plt.subplots(1,2)
    sns.distplot(original_data, ax=ax[0], color='y')
    ax[0].set_title("Original Data")
    if scaled_data:
        sns.distplot(scaled_data, ax=ax[1])
        ax[1].set_title("Scaled data")
    plt.show()

In [19]:
# fit and evaluate a model
def evaluate_model(optimzer_choice, trainX, trainy, testX, testy, act_fun, epochs=200, patience=20):

    _verbose, epochs, batch_size, patience = 1, epochs, 128, patience

    n_timesteps, n_features, n_outputs = SLIDING_WINDOW_LENGTH, SENSOR_CHANNELS, NUM_CLASSES

    #input 1 window 1;
    inputs1 = Input(shape=(n_timesteps,n_features), name='x1')
    convs1_1 = Conv1D(filters=8, kernel_size=2, strides=1, kernel_regularizer='l2', padding='same', activation=act_fun)(inputs1)
    pools1_1 = MaxPooling1D(pool_size=2, strides=1)(convs1_1)
    drops1_1 = Dropout(0.5)(pools1_1)
    convs1_2 = Conv1D(filters=18, kernel_size=2, strides=1, kernel_regularizer='l2', padding='same', activation=act_fun)(drops1_1)
    pools1_2 = MaxPooling1D(pool_size=2, strides=1)(convs1_2)
    drops1_2 = Dropout(0.5)(pools1_2)
    convs1_3 = Conv1D(filters=36, kernel_size=2, strides=1, kernel_regularizer='l2', padding='same', activation=act_fun)(drops1_2)
    pools1_3 = MaxPooling1D(pool_size=2, strides=1)(convs1_3)
    drops1_3 = Dropout(0.5)(pools1_3)

    
    #input 2 windows 2;
    inputs2 = Input(shape=(n_timesteps,n_features), name='x2')
    convs2_1 = Conv1D(filters=8, kernel_size=2, strides=1, kernel_regularizer='l2', padding='same', activation=act_fun)(inputs2)
    pools2_1 = MaxPooling1D(pool_size=2, strides=1)(convs2_1)
    drops2_1 = Dropout(0.5)(pools2_1)
    convs2_2 = Conv1D(filters=18, kernel_size=2, strides=1, kernel_regularizer='l2', padding='same', activation=act_fun)(drops2_1)
    pools2_2 = MaxPooling1D(pool_size=2, strides=1)(convs2_2)
    drops2_2 = Dropout(0.5)(pools2_2)
    convs2_3 = Conv1D(filters=36, kernel_size=2, strides=1, kernel_regularizer='l2', padding='same', activation=act_fun)(drops2_2)
    pools2_3 = MaxPooling1D(pool_size=2, strides=1)(convs2_3)
    drops2_3 = Dropout(0.5)(pools2_3)


    #input 2 windows 2;
    inputs3 = Input(shape=(n_timesteps,n_features), name='x3')
    convs3_1 = Conv1D(filters=8, kernel_size=2, strides=1, kernel_regularizer='l2', padding='same', activation=act_fun)(inputs3)
    pools3_1 = MaxPooling1D(pool_size=2, strides=1)(convs3_1)
    drops3_1 = Dropout(0.5)(pools3_1)
    convs3_2 = Conv1D(filters=18, kernel_size=2, strides=1, kernel_regularizer='l2', padding='same', activation=act_fun)(drops3_1)
    pools3_2 = MaxPooling1D(pool_size=2, strides=1)(convs3_2)
    drops3_2 = Dropout(0.5)(pools3_2)
    convs3_3 = Conv1D(filters=36, kernel_size=2, strides=1, kernel_regularizer='l2', padding='same', activation=act_fun)(drops3_2)
    pools3_3 = MaxPooling1D(pool_size=2, strides=1)(convs3_3)
    drops3_3 = Dropout(0.5)(pools3_3)


    #merged = concatenate([lstm_1_1, lstm_2_1], axis=1)
    merged = concatenate([drops1_3, drops2_3, drops3_3], axis=1)
    lstm_1 = LSTM(48, return_sequences=False, kernel_regularizer='l2')(merged)
    drops3_1 = Dropout(0.5)(lstm_1)
    output = Dense(n_outputs, activation='softmax')(drops3_1)
    
    model = Model(inputs=[inputs1, inputs2, inputs3], outputs=output)
    opt = optimizers.Adam(learning_rate=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    #print(model.summary())
  
    es = EarlyStopping(monitor='loss', patience=patience, verbose=True)
    cp = ModelCheckpoint(filepath='benchmark_cp', monitor='loss', save_weights_only=True)
    #model.fit({'x1':trainX[:-1], 'x2':trainX[1:]}, trainy[1:], validation_split=0.2, epochs=epochs, batch_size=batch_size, verbose=_verbose, callbacks=[es, cp])
    history = model.fit({'x1':trainX[:-2], 'x2':trainX[1:-1], 'x3':trainX[2:]}, trainy[2:], epochs=epochs, batch_size=batch_size, verbose=_verbose, callbacks=[cp])
    model.load_weights('benchmark_cp')
    #plot_model(model, to_file='3CNN+LSTM.png', show_shapes=True, show_layer_names=True)
    return model, history

In [20]:
# run an experiment
def run_experiment(repeats=1, epochs=200, patience=20):
    # load data
    trainX, trainy, testX, testy = load_dataset()
    
    # prev_trainX = trainX
    scaler = StandardScaler() #check if the scaler if this impact the ReLu. https://mlfromscratch.com/activation-functions-explained/#/
    #scaler = MinMaxScaler()
    trainX = scaler.fit_transform(trainX)
    testX = scaler.fit_transform(testX)
    
    # Sensor data is segmented using a sliding window mechanism
    trainX, trainy = har_sliding_window(trainX, trainy, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
    testX, testy = har_sliding_window(testX, testy, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)

    # Data is reshaped
    trainX = trainX.reshape((-1, SLIDING_WINDOW_LENGTH, SENSOR_CHANNELS)) # for input to Conv1D
    testX = testX.reshape((-1, SLIDING_WINDOW_LENGTH, SENSOR_CHANNELS)) # for input to Conv1D

    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    
    # process to throw the first row
    trainy = trainy[:,1:13]
    testy = testy[:,1:13]

    testy = np.argmax(testy, axis=1)
    testy = testy + 1

    # choice for activation fucntion
    act_fun = ['tanh']#, 'relu', 'sigmoid', 'softplus', 'softsign', 'selu', 'elu']
    Optimizer_choice = 1
    max_loop = 5
    # configure the model
    print(trainX.shape)
    print(testX.shape)
    #for act in act_fun: #loop for activation function
    act = 'tanh'
    history_list = []
    for r in range(max_loop): #loop 10
        model_g, history = evaluate_model(r, trainX, trainy, testX, testy, act_fun=act, epochs=epochs, patience=patience)

        y_pred = model_g.predict([testX,testX,testX])
        y_predict = np.argmax(y_pred, axis=1)
        y_predict= y_predict + 1 

        print("Activation: {} Loop {}: accuracy: {:.2f}".format(act, r+1, accuracy_score(testy, y_predict)*100))
        print(confusion_matrix(testy, y_predict))
        print(classification_report(testy, y_predict))
        print("====================================================")
    
        history_list.append(history)

        model_dir = 'proposed_final_' + str(r)
        if not path.exists(model_dir):
            os.mkdir(model_dir) 
        model_g.save(model_dir)
    
    return history_list
    
# run the experiment
history_list = run_experiment(epochs=500, patience=30)

Train set:(554294, 6)(554294,)
Test set:(261320, 6)(261320,)
(8659, 128, 6)
(4082, 128, 6)
Epoch 1/500
68/68 [==============================] - 2s 27ms/step - loss: 3.2528 - accuracy: 0.3394
Epoch 2/500
68/68 [==============================] - 2s 27ms/step - loss: 2.2262 - accuracy: 0.4579
Epoch 3/500
68/68 [==============================] - 2s 28ms/step - loss: 1.7991 - accuracy: 0.5383
Epoch 4/500
68/68 [==============================] - 2s 28ms/step - loss: 1.5788 - accuracy: 0.5958
Epoch 5/500
68/68 [==============================] - 2s 27ms/step - loss: 1.3519 - accuracy: 0.6697
Epoch 6/500
68/68 [==============================] - 2s 28ms/step - loss: 1.2619 - accuracy: 0.6942
Epoch 7/500
68/68 [==============================] - 2s 27ms/step - loss: 1.1498 - accuracy: 0.7225
Epoch 8/500
68/68 [==============================] - 2s 28ms/step - loss: 1.1181 - accuracy: 0.7177
Epoch 9/500
68/68 [==============================] - 2s 27ms/step - loss: 1.0639 - accuracy: 0.7266
Epoch 10/

68/68 [==============================] - 2s 28ms/step - loss: 0.5326 - accuracy: 0.8538
Epoch 161/500
68/68 [==============================] - 2s 28ms/step - loss: 0.5241 - accuracy: 0.8584
Epoch 162/500
68/68 [==============================] - 2s 27ms/step - loss: 0.5209 - accuracy: 0.8531
Epoch 163/500
68/68 [==============================] - 2s 27ms/step - loss: 0.5085 - accuracy: 0.8524
Epoch 164/500
68/68 [==============================] - 2s 28ms/step - loss: 0.5277 - accuracy: 0.8526
Epoch 165/500
68/68 [==============================] - 2s 28ms/step - loss: 0.5415 - accuracy: 0.8451
Epoch 166/500
68/68 [==============================] - 2s 26ms/step - loss: 0.5098 - accuracy: 0.8575
Epoch 167/500
68/68 [==============================] - 2s 28ms/step - loss: 0.4985 - accuracy: 0.8562
Epoch 168/500
68/68 [==============================] - 2s 26ms/step - loss: 0.5131 - accuracy: 0.8511
Epoch 169/500
68/68 [==============================] - 2s 27ms/step - loss: 0.5197 - accuracy: 0

68/68 [==============================] - 2s 27ms/step - loss: 0.3875 - accuracy: 0.8831
Epoch 321/500
68/68 [==============================] - 2s 27ms/step - loss: 0.3631 - accuracy: 0.8870
Epoch 322/500
68/68 [==============================] - 2s 27ms/step - loss: 0.3677 - accuracy: 0.8908
Epoch 323/500
68/68 [==============================] - 2s 27ms/step - loss: 0.3688 - accuracy: 0.8883
Epoch 324/500
68/68 [==============================] - 2s 27ms/step - loss: 0.3631 - accuracy: 0.8901
Epoch 325/500
68/68 [==============================] - 2s 27ms/step - loss: 0.4062 - accuracy: 0.8798
Epoch 326/500
68/68 [==============================] - 2s 28ms/step - loss: 0.3805 - accuracy: 0.8851
Epoch 327/500
68/68 [==============================] - 2s 28ms/step - loss: 0.3771 - accuracy: 0.8897
Epoch 328/500
68/68 [==============================] - 2s 28ms/step - loss: 0.3691 - accuracy: 0.8884
Epoch 329/500
68/68 [==============================] - 2s 28ms/step - loss: 0.3589 - accuracy: 0

68/68 [==============================] - 2s 28ms/step - loss: 0.2925 - accuracy: 0.9102
Epoch 481/500
68/68 [==============================] - 2s 28ms/step - loss: 0.2870 - accuracy: 0.9111
Epoch 482/500
68/68 [==============================] - 2s 28ms/step - loss: 0.2931 - accuracy: 0.9065
Epoch 483/500
68/68 [==============================] - 2s 27ms/step - loss: 0.3083 - accuracy: 0.9064
Epoch 484/500
68/68 [==============================] - 2s 28ms/step - loss: 0.2946 - accuracy: 0.9105
Epoch 485/500
68/68 [==============================] - 2s 28ms/step - loss: 0.2829 - accuracy: 0.9144
Epoch 486/500
68/68 [==============================] - 2s 28ms/step - loss: 0.2980 - accuracy: 0.9093
Epoch 487/500
68/68 [==============================] - 2s 28ms/step - loss: 0.3085 - accuracy: 0.9067
Epoch 488/500
68/68 [==============================] - 2s 27ms/step - loss: 0.2938 - accuracy: 0.9093
Epoch 489/500
68/68 [==============================] - 2s 28ms/step - loss: 0.2944 - accuracy: 0

Epoch 45/500
68/68 [==============================] - 2s 27ms/step - loss: 0.8056 - accuracy: 0.7894
Epoch 46/500
68/68 [==============================] - 2s 27ms/step - loss: 0.8419 - accuracy: 0.7833
Epoch 47/500
68/68 [==============================] - 2s 27ms/step - loss: 0.7979 - accuracy: 0.7917
Epoch 48/500
68/68 [==============================] - 2s 28ms/step - loss: 0.7412 - accuracy: 0.8063
Epoch 49/500
68/68 [==============================] - 2s 27ms/step - loss: 0.7047 - accuracy: 0.8163
Epoch 50/500
68/68 [==============================] - 2s 27ms/step - loss: 0.7406 - accuracy: 0.8055
Epoch 51/500
68/68 [==============================] - 2s 27ms/step - loss: 0.6916 - accuracy: 0.8188
Epoch 52/500
68/68 [==============================] - 2s 27ms/step - loss: 0.6565 - accuracy: 0.8251
Epoch 53/500
68/68 [==============================] - 2s 27ms/step - loss: 0.6411 - accuracy: 0.8309
Epoch 54/500
68/68 [==============================] - 2s 28ms/step - loss: 0.6534 - accurac

...

68/68 [==============================] - 2s 28ms/step - loss: 0.3680 - accuracy: 0.8875
Epoch 376/500
68/68 [==============================] - 2s 27ms/step - loss: 0.3577 - accuracy: 0.8876
Epoch 377/500
68/68 [==============================] - 2s 28ms/step - loss: 0.3621 - accuracy: 0.8897
Epoch 378/500
68/68 [==============================] - 2s 27ms/step - loss: 0.3584 - accuracy: 0.8898
Epoch 379/500
68/68 [==============================] - 2s 28ms/step - loss: 0.3518 - accuracy: 0.8947
Epoch 380/500
68/68 [==============================] - 2s 28ms/step - loss: 0.3497 - accuracy: 0.8895
Epoch 381/500
68/68 [==============================] - 2s 27ms/step - loss: 0.3599 - accuracy: 0.8906
Epoch 382/500
68/68 [==============================] - 2s 28ms/step - loss: 0.4063 - accuracy: 0.8798
Epoch 383/500
68/68 [==============================] - 2s 28ms/step - loss: 0.4119 - accuracy: 0.8770
Epoch 384/500
68/68 [==============================] - 2s 27ms/step - loss: 0.4343 - accuracy: 0

68/68 [==============================] - 2s 27ms/step - loss: 0.9354 - accuracy: 0.7512
Epoch 20/500
68/68 [==============================] - 2s 27ms/step - loss: 0.9033 - accuracy: 0.7611
Epoch 21/500
68/68 [==============================] - 2s 27ms/step - loss: 0.8761 - accuracy: 0.7726
Epoch 22/500
68/68 [==============================] - 2s 27ms/step - loss: 0.8642 - accuracy: 0.7754
Epoch 23/500
68/68 [==============================] - 2s 27ms/step - loss: 0.8306 - accuracy: 0.7888
Epoch 24/500
68/68 [==============================] - 2s 28ms/step - loss: 0.9019 - accuracy: 0.7750
Epoch 25/500
68/68 [==============================] - 2s 27ms/step - loss: 0.8337 - accuracy: 0.7905
Epoch 26/500
68/68 [==============================] - 2s 28ms/step - loss: 0.8205 - accuracy: 0.7947
Epoch 27/500
68/68 [==============================] - 2s 27ms/step - loss: 0.7660 - accuracy: 0.8109
Epoch 28/500
68/68 [==============================] - 2s 27ms/step - loss: 0.7662 - accuracy: 0.8057
Epo

68/68 [==============================] - 2s 27ms/step - loss: 0.4652 - accuracy: 0.8707
Epoch 181/500
68/68 [==============================] - 2s 27ms/step - loss: 0.4265 - accuracy: 0.8798
Epoch 182/500
68/68 [==============================] - 2s 27ms/step - loss: 0.4188 - accuracy: 0.8802
Epoch 183/500
68/68 [==============================] - 2s 27ms/step - loss: 0.4399 - accuracy: 0.8692
Epoch 184/500
68/68 [==============================] - 2s 27ms/step - loss: 0.4005 - accuracy: 0.8847
Epoch 185/500
68/68 [==============================] - 2s 27ms/step - loss: 0.4053 - accuracy: 0.8800
Epoch 186/500
68/68 [==============================] - 2s 27ms/step - loss: 0.4218 - accuracy: 0.8789
Epoch 187/500
68/68 [==============================] - 2s 27ms/step - loss: 0.4121 - accuracy: 0.8823
Epoch 188/500
68/68 [==============================] - 2s 27ms/step - loss: 0.3874 - accuracy: 0.8907
Epoch 189/500
68/68 [==============================] - 2s 27ms/step - loss: 0.4355 - accuracy: 0

68/68 [==============================] - 2s 26ms/step - loss: 0.3374 - accuracy: 0.8992
Epoch 341/500
68/68 [==============================] - 2s 27ms/step - loss: 0.3255 - accuracy: 0.9035
Epoch 342/500
68/68 [==============================] - 2s 27ms/step - loss: 0.3665 - accuracy: 0.8901
Epoch 343/500
68/68 [==============================] - 2s 27ms/step - loss: 0.3689 - accuracy: 0.8907
Epoch 344/500
68/68 [==============================] - 2s 28ms/step - loss: 0.3347 - accuracy: 0.8995
Epoch 345/500
68/68 [==============================] - 2s 27ms/step - loss: 0.3256 - accuracy: 0.9002
Epoch 346/500
68/68 [==============================] - 2s 27ms/step - loss: 0.3241 - accuracy: 0.9029
Epoch 347/500
68/68 [==============================] - 2s 28ms/step - loss: 0.3191 - accuracy: 0.9038
Epoch 348/500
68/68 [==============================] - 2s 27ms/step - loss: 0.3173 - accuracy: 0.9031
Epoch 349/500
68/68 [==============================] - 2s 27ms/step - loss: 0.3315 - accuracy: 0

68/68 [==============================] - 2s 26ms/step - loss: 0.2771 - accuracy: 0.9129
Activation: tanh Loop 5: accuracy: 90.45
[[523  48   3   0   2   0   0   1   0   0   0   3]
 [ 20 534  12   0   1   0   0   1   0   0   0   0]
 [  1   3 531   0   0   0   0   0   0   0   0   2]
 [  0   0   0 580  83   0   2   1   0   2   0   0]
 [  0   1   0  44 655   0   0   2   1   0   5   0]
 [  0   0   0   0   0 714   0   0   3   2   0   0]
 [  0   0   0   5   8   0  22   2   0   2   9   1]
 [  0   0   0   3   6   0   0  30   0   0   0   1]
 [  0   0   0   4   2   7   0   0  33   0  13   0]
 [  0   0   0  11   0   3   1   0   0  29   0   4]
 [  0   0   0   4   3   4   0   0  25   0  19   0]
 [  1   0   0   2   3   4   0   1   0  17   1  22]]
              precision    recall  f1-score   support

           1       0.96      0.90      0.93       580
           2       0.91      0.94      0.93       568
           3       0.97      0.99      0.98       537
           4       0.89      0.87      0.

In [21]:
# run an experiment
def run_experiment(repeats=1, epochs=200, patience=20):
    # load data
    trainX, trainy, testX, testy = load_dataset()
    
    # prev_trainX = trainX
    scaler = StandardScaler() #check if the scaler if this impact the ReLu. https://mlfromscratch.com/activation-functions-explained/#/
    #scaler = MinMaxScaler()
    trainX = scaler.fit_transform(trainX)
    testX = scaler.fit_transform(testX)
    
    # Sensor data is segmented using a sliding window mechanism
    trainX, trainy = har_sliding_window(trainX, trainy, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
    testX, testy = har_sliding_window(testX, testy, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)

    # Data is reshaped
    trainX = trainX.reshape((-1, SLIDING_WINDOW_LENGTH, SENSOR_CHANNELS)) # for input to Conv1D
    testX = testX.reshape((-1, SLIDING_WINDOW_LENGTH, SENSOR_CHANNELS)) # for input to Conv1D

    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    
    # process to throw the first row
    trainy = trainy[:,1:13]
    testy = testy[:,1:13]

    testy = np.argmax(testy, axis=1)
    testy = testy + 1

    # choice for activation fucntion
    act_fun = ['tanh']#, 'relu', 'sigmoid', 'softplus', 'softsign', 'selu', 'elu']
    Optimizer_choice = 1
    max_loop = 30
    # configure the model
    print(trainX.shape)
    print(testX.shape)
    #for act in act_fun: #loop for activation function
    act = 'tanh'
    history_list = []
    for r in range(5,max_loop): 
        model_g, history = evaluate_model(r, trainX, trainy, testX, testy, act_fun=act, epochs=epochs, patience=patience)

        y_pred = model_g.predict([testX,testX,testX])
        y_predict = np.argmax(y_pred, axis=1)
        y_predict= y_predict + 1 

        print("Activation: {} Loop {}: accuracy: {:.2f}".format(act, r+1, accuracy_score(testy, y_predict)*100))
        print(confusion_matrix(testy, y_predict))
        print(classification_report(testy, y_predict))
        print("====================================================")
    
        history_list.append(history)

        model_dir = 'proposed_final_' + str(r)
        if not path.exists(model_dir):
            os.mkdir(model_dir) 
        model_g.save(model_dir)
    
    return history_list
    
# run the experiment
history_list2 = run_experiment(epochs=500, patience=30)

Train set:(554294, 6)(554294,)
Test set:(261320, 6)(261320,)
(8659, 128, 6)
(4082, 128, 6)
Epoch 1/500
68/68 [==============================] - 2s 28ms/step - loss: 3.2080 - accuracy: 0.3544
Epoch 2/500
68/68 [==============================] - 2s 27ms/step - loss: 2.1863 - accuracy: 0.4753
Epoch 3/500
68/68 [==============================] - 2s 28ms/step - loss: 1.7535 - accuracy: 0.5582
Epoch 4/500
68/68 [==============================] - 2s 27ms/step - loss: 1.5371 - accuracy: 0.6023
Epoch 5/500
68/68 [==============================] - 2s 28ms/step - loss: 1.4252 - accuracy: 0.6244
Epoch 6/500
68/68 [==============================] - 2s 28ms/step - loss: 1.3364 - accuracy: 0.6386
Epoch 7/500
68/68 [==============================] - 2s 27ms/step - loss: 1.3439 - accuracy: 0.6366
Epoch 8/500
68/68 [==============================] - 2s 27ms/step - loss: 1.2285 - accuracy: 0.6711
Epoch 9/500
68/68 [==============================] - 2s 28ms/step - loss: 1.1883 - accuracy: 0.6718
Epoch 10/

68/68 [==============================] - 2s 27ms/step - loss: 0.7537 - accuracy: 0.8024
Epoch 162/500
68/68 [==============================] - 2s 27ms/step - loss: 0.6161 - accuracy: 0.8348
Epoch 163/500
68/68 [==============================] - 2s 28ms/step - loss: 0.5796 - accuracy: 0.8391
Epoch 164/500
68/68 [==============================] - 2s 26ms/step - loss: 0.5252 - accuracy: 0.8488
Epoch 165/500
68/68 [==============================] - 2s 27ms/step - loss: 0.5303 - accuracy: 0.8486
Epoch 166/500
68/68 [==============================] - 2s 26ms/step - loss: 0.4921 - accuracy: 0.8556
Epoch 167/500
68/68 [==============================] - 2s 27ms/step - loss: 0.4843 - accuracy: 0.8573
Epoch 168/500
68/68 [==============================] - 2s 26ms/step - loss: 0.4546 - accuracy: 0.8713
Epoch 169/500
68/68 [==============================] - 2s 27ms/step - loss: 0.4624 - accuracy: 0.8660
Epoch 170/500
68/68 [==============================] - 2s 28ms/step - loss: 0.4731 - accuracy: 0

68/68 [==============================] - 2s 27ms/step - loss: 0.3438 - accuracy: 0.8992
Epoch 322/500
68/68 [==============================] - 2s 28ms/step - loss: 0.3491 - accuracy: 0.8989
Epoch 323/500
68/68 [==============================] - 2s 27ms/step - loss: 0.3430 - accuracy: 0.8971
Epoch 324/500
68/68 [==============================] - 2s 27ms/step - loss: 0.3317 - accuracy: 0.9000
Epoch 325/500
68/68 [==============================] - 2s 27ms/step - loss: 0.4932 - accuracy: 0.8549
Epoch 326/500
68/68 [==============================] - 2s 27ms/step - loss: 0.4890 - accuracy: 0.8608
Epoch 327/500
68/68 [==============================] - 2s 27ms/step - loss: 0.4330 - accuracy: 0.8773
Epoch 328/500
68/68 [==============================] - 2s 26ms/step - loss: 0.3896 - accuracy: 0.8878
Epoch 329/500
68/68 [==============================] - 2s 27ms/step - loss: 0.3648 - accuracy: 0.8926
Epoch 330/500
68/68 [==============================] - 2s 26ms/step - loss: 0.3613 - accuracy: 0

68/68 [==============================] - 2s 27ms/step - loss: 0.2824 - accuracy: 0.9152
Epoch 482/500
68/68 [==============================] - 2s 27ms/step - loss: 0.2825 - accuracy: 0.9115
Epoch 483/500
68/68 [==============================] - 2s 28ms/step - loss: 0.3109 - accuracy: 0.9048
Epoch 484/500
68/68 [==============================] - 2s 28ms/step - loss: 0.3241 - accuracy: 0.9011
Epoch 485/500
68/68 [==============================] - 2s 27ms/step - loss: 0.3148 - accuracy: 0.9042
Epoch 486/500
68/68 [==============================] - 2s 26ms/step - loss: 0.3040 - accuracy: 0.9085
Epoch 487/500
68/68 [==============================] - 2s 27ms/step - loss: 0.2808 - accuracy: 0.9167
Epoch 488/500
68/68 [==============================] - 2s 27ms/step - loss: 0.2931 - accuracy: 0.9106
Epoch 489/500
68/68 [==============================] - 2s 28ms/step - loss: 0.2837 - accuracy: 0.9143
Epoch 490/500
68/68 [==============================] - 2s 27ms/step - loss: 0.2874 - accuracy: 0

Epoch 46/500
68/68 [==============================] - 2s 27ms/step - loss: 0.6875 - accuracy: 0.8196
Epoch 47/500
68/68 [==============================] - 2s 28ms/step - loss: 0.6731 - accuracy: 0.8238
Epoch 48/500
68/68 [==============================] - 2s 28ms/step - loss: 0.6871 - accuracy: 0.8206
Epoch 49/500
68/68 [==============================] - 2s 27ms/step - loss: 0.6271 - accuracy: 0.8324
Epoch 50/500
68/68 [==============================] - 2s 27ms/step - loss: 0.7278 - accuracy: 0.8118
Epoch 51/500
68/68 [==============================] - 2s 28ms/step - loss: 1.1480 - accuracy: 0.6991
Epoch 52/500
68/68 [==============================] - 2s 27ms/step - loss: 0.8774 - accuracy: 0.7831
Epoch 53/500
68/68 [==============================] - 2s 27ms/step - loss: 0.8579 - accuracy: 0.7749
Epoch 54/500
68/68 [==============================] - 2s 27ms/step - loss: 0.7836 - accuracy: 0.8078
Epoch 55/500
68/68 [==============================] - 2s 27ms/step - loss: 0.7669 - accurac

...

68/68 [==============================] - 2s 28ms/step - loss: 0.6020 - accuracy: 0.8369
Epoch 332/500
68/68 [==============================] - 2s 28ms/step - loss: 0.7119 - accuracy: 0.8081
Epoch 333/500
68/68 [==============================] - 2s 28ms/step - loss: 0.6306 - accuracy: 0.8310
Epoch 334/500
68/68 [==============================] - 2s 28ms/step - loss: 0.6124 - accuracy: 0.8372
Epoch 335/500
68/68 [==============================] - 2s 28ms/step - loss: 0.5757 - accuracy: 0.8414
Epoch 336/500
68/68 [==============================] - 2s 28ms/step - loss: 0.5881 - accuracy: 0.8362
Epoch 337/500
68/68 [==============================] - 2s 28ms/step - loss: 0.5617 - accuracy: 0.8443
Epoch 338/500
68/68 [==============================] - 2s 28ms/step - loss: 0.5863 - accuracy: 0.8387
Epoch 339/500
68/68 [==============================] - 2s 28ms/step - loss: 0.5529 - accuracy: 0.8449
Epoch 340/500
68/68 [==============================] - 2s 27ms/step - loss: 0.5408 - accuracy: 0

68/68 [==============================] - 2s 28ms/step - loss: 0.4266 - accuracy: 0.8833
Epoch 492/500
68/68 [==============================] - 2s 29ms/step - loss: 0.4650 - accuracy: 0.8699
Epoch 493/500
68/68 [==============================] - 2s 28ms/step - loss: 0.4173 - accuracy: 0.8854
Epoch 494/500
68/68 [==============================] - 2s 28ms/step - loss: 0.4069 - accuracy: 0.8870
Epoch 495/500
68/68 [==============================] - 2s 28ms/step - loss: 0.4131 - accuracy: 0.8875
Epoch 496/500
68/68 [==============================] - 2s 28ms/step - loss: 0.4279 - accuracy: 0.8778
Epoch 497/500
68/68 [==============================] - 2s 28ms/step - loss: 0.4471 - accuracy: 0.8763
Epoch 498/500
68/68 [==============================] - 2s 28ms/step - loss: 0.4853 - accuracy: 0.8651
Epoch 499/500
68/68 [==============================] - 2s 28ms/step - loss: 0.4283 - accuracy: 0.8800
Epoch 500/500
68/68 [==============================] - 2s 27ms/step - loss: 0.4154 - accuracy: 0

68/68 [==============================] - 2s 28ms/step - loss: 0.6735 - accuracy: 0.8147
Epoch 57/500
68/68 [==============================] - 2s 28ms/step - loss: 0.7240 - accuracy: 0.7954
Epoch 58/500
68/68 [==============================] - 2s 28ms/step - loss: 0.6645 - accuracy: 0.8190
Epoch 59/500
68/68 [==============================] - 2s 29ms/step - loss: 0.6179 - accuracy: 0.8335
Epoch 60/500
68/68 [==============================] - 2s 29ms/step - loss: 0.6018 - accuracy: 0.8413 0s
Epoch 61/500
68/68 [==============================] - 2s 28ms/step - loss: 0.6010 - accuracy: 0.8369
Epoch 62/500
68/68 [==============================] - 2s 29ms/step - loss: 0.6031 - accuracy: 0.8361
Epoch 63/500
68/68 [==============================] - 2s 28ms/step - loss: 0.5611 - accuracy: 0.8484
Epoch 64/500
68/68 [==============================] - 2s 28ms/step - loss: 0.5407 - accuracy: 0.8526
Epoch 65/500
68/68 [==============================] - 2s 28ms/step - loss: 0.5339 - accuracy: 0.8533


68/68 [==============================] - 2s 28ms/step - loss: 0.3651 - accuracy: 0.8871
Epoch 217/500
68/68 [==============================] - 2s 28ms/step - loss: 0.3834 - accuracy: 0.8832
Epoch 218/500
68/68 [==============================] - 2s 27ms/step - loss: 0.4073 - accuracy: 0.8786
Epoch 219/500
68/68 [==============================] - 2s 28ms/step - loss: 0.4189 - accuracy: 0.8761
Epoch 220/500
68/68 [==============================] - 2s 28ms/step - loss: 0.3995 - accuracy: 0.8777
Epoch 221/500
68/68 [==============================] - 2s 28ms/step - loss: 0.3709 - accuracy: 0.8880
Epoch 222/500
68/68 [==============================] - 2s 28ms/step - loss: 0.3643 - accuracy: 0.8897
Epoch 223/500
68/68 [==============================] - 2s 28ms/step - loss: 0.3597 - accuracy: 0.8919
Epoch 224/500
68/68 [==============================] - 2s 27ms/step - loss: 0.3522 - accuracy: 0.8941
Epoch 225/500
68/68 [==============================] - 2s 28ms/step - loss: 0.4939 - accuracy: 0

68/68 [==============================] - 2s 27ms/step - loss: 0.3299 - accuracy: 0.9010
Epoch 377/500
68/68 [==============================] - 2s 28ms/step - loss: 0.3665 - accuracy: 0.8839
Epoch 378/500
68/68 [==============================] - 2s 27ms/step - loss: 0.3366 - accuracy: 0.8978
Epoch 379/500
68/68 [==============================] - 2s 28ms/step - loss: 0.3283 - accuracy: 0.9024
Epoch 380/500
68/68 [==============================] - 2s 27ms/step - loss: 0.3082 - accuracy: 0.9056
Epoch 381/500
68/68 [==============================] - 2s 28ms/step - loss: 0.3098 - accuracy: 0.9075
Epoch 382/500
68/68 [==============================] - 2s 28ms/step - loss: 0.3038 - accuracy: 0.9064
Epoch 383/500
68/68 [==============================] - 2s 28ms/step - loss: 0.3066 - accuracy: 0.9038
Epoch 384/500
68/68 [==============================] - 2s 29ms/step - loss: 0.3108 - accuracy: 0.9026
Epoch 385/500
68/68 [==============================] - 2s 28ms/step - loss: 0.3235 - accuracy: 0

In [39]:
for i in range(5):
    file_loss = 'hist_loss' + str(i) + '.txt'
    file_acc = 'hist_acc' + str(i) + '.txt'
    f = open(file_loss, "w")
    for e in history_list[i].history['loss']:
        f.write(str(e) + '\n')
    f.close()
    f = open(file_acc, "w")
    for e in history_list[i].history['accuracy']:
        f.write(str(e) + '\n')
    f.close()

In [40]:
for i in range(25):
    file_loss = 'hist_loss' + str(i+5) + '.txt'
    file_acc = 'hist_acc' + str(i+5) + '.txt'
    f = open(file_loss, "w")
    for e in history_list2[i].history['loss']:
        f.write(str(e) + '\n')
    f.close()
    f = open(file_acc, "w")
    for e in history_list2[i].history['accuracy']:
        f.write(str(e) + '\n')
    f.close()

In [43]:
n_timesteps, n_features, n_outputs = SLIDING_WINDOW_LENGTH, SENSOR_CHANNELS, NUM_CLASSES
act_fun = 'tanh'
#input 1 window 1;
inputs1 = Input(shape=(n_timesteps,n_features), name='x1')
convs1_1 = Conv1D(filters=8, kernel_size=2, strides=1, kernel_regularizer='l2', padding='same', activation=act_fun)(inputs1)
pools1_1 = MaxPooling1D(pool_size=2, strides=1)(convs1_1)
drops1_1 = Dropout(0.5)(pools1_1)
convs1_2 = Conv1D(filters=18, kernel_size=2, strides=1, kernel_regularizer='l2', padding='same', activation=act_fun)(drops1_1)
pools1_2 = MaxPooling1D(pool_size=2, strides=1)(convs1_2)
drops1_2 = Dropout(0.5)(pools1_2)
convs1_3 = Conv1D(filters=36, kernel_size=2, strides=1, kernel_regularizer='l2', padding='same', activation=act_fun)(drops1_2)
pools1_3 = MaxPooling1D(pool_size=2, strides=1)(convs1_3)
drops1_3 = Dropout(0.5)(pools1_3)


#input 2 windows 2;
inputs2 = Input(shape=(n_timesteps,n_features), name='x2')
convs2_1 = Conv1D(filters=8, kernel_size=2, strides=1, kernel_regularizer='l2', padding='same', activation=act_fun)(inputs2)
pools2_1 = MaxPooling1D(pool_size=2, strides=1)(convs2_1)
drops2_1 = Dropout(0.5)(pools2_1)
convs2_2 = Conv1D(filters=18, kernel_size=2, strides=1, kernel_regularizer='l2', padding='same', activation=act_fun)(drops2_1)
pools2_2 = MaxPooling1D(pool_size=2, strides=1)(convs2_2)
drops2_2 = Dropout(0.5)(pools2_2)
convs2_3 = Conv1D(filters=36, kernel_size=2, strides=1, kernel_regularizer='l2', padding='same', activation=act_fun)(drops2_2)
pools2_3 = MaxPooling1D(pool_size=2, strides=1)(convs2_3)
drops2_3 = Dropout(0.5)(pools2_3)


#input 2 windows 2;
inputs3 = Input(shape=(n_timesteps,n_features), name='x3')
convs3_1 = Conv1D(filters=8, kernel_size=2, strides=1, kernel_regularizer='l2', padding='same', activation=act_fun)(inputs3)
pools3_1 = MaxPooling1D(pool_size=2, strides=1)(convs3_1)
drops3_1 = Dropout(0.5)(pools3_1)
convs3_2 = Conv1D(filters=18, kernel_size=2, strides=1, kernel_regularizer='l2', padding='same', activation=act_fun)(drops3_1)
pools3_2 = MaxPooling1D(pool_size=2, strides=1)(convs3_2)
drops3_2 = Dropout(0.5)(pools3_2)
convs3_3 = Conv1D(filters=36, kernel_size=2, strides=1, kernel_regularizer='l2', padding='same', activation=act_fun)(drops3_2)
pools3_3 = MaxPooling1D(pool_size=2, strides=1)(convs3_3)
drops3_3 = Dropout(0.5)(pools3_3)


#merged = concatenate([lstm_1_1, lstm_2_1], axis=1)
merged = concatenate([drops1_3, drops2_3, drops3_3], axis=1)
lstm_1 = LSTM(48, return_sequences=False, kernel_regularizer='l2')(merged)
drops3_1 = Dropout(0.5)(lstm_1)
output = Dense(n_outputs, activation='softmax')(drops3_1)

pred_model = Model(inputs=[inputs1, inputs2, inputs3], outputs=output)

In [ ]:
trainX, trainy, testX, testy = load_dataset()
scaler = StandardScaler() #check if the scaler if this impact the ReLu. https://mlfromscratch.com/activation-functions-explained/#/
testX = scaler.fit_transform(testX)
testX, testy = har_sliding_window(testX, testy, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
testX = testX.reshape((-1, SLIDING_WINDOW_LENGTH, SENSOR_CHANNELS)) # for input to Conv1D
# one hot encode y
testy = to_categorical(testy)
# process to throw the first row
testy = testy[:,1:13]
testy = np.argmax(testy, axis=1)
testy = testy + 1

f = open('model_accuracies.txt', "w") 
for i in range(0,30):
    model_file = 'proposed_final_' + str(i)
    pred_model = tf.keras.models.load_model(model_file)
    
    y_pred = pred_model.predict([testX,testX,testX])
    y_predict = np.argmax(y_pred, axis=1)
    y_predict= y_predict + 1 
    print('Model ' + str(i+1))
    accuracy = accuracy_score(testy, y_predict)
    print(accuracy)
    print(confusion_matrix(testy, y_predict))
    print(classification_report(testy, y_predict))
    f.write(str(accuracy) + '\n')
f.close()

Train set:(554294, 6)(554294,)
Test set:(261320, 6)(261320,)
Model 1
0.8860852523272905
[[555  16   3   0   2   0   1   0   0   1   1   1]
 [ 27 523  10   0   3   0   2   0   0   0   2   1]
 [  3   6 516   0   1   0   7   0   0   1   3   0]
 [  0   0   0 577  84   0   2   2   0   2   1   0]
 [  0   1   0 126 569   1   3   0   0   0   6   2]
 [  0   0   0   0   0 715   0   0   0   3   1   0]
 [  0   0   0   8   7   0  28   1   0   2   2   1]
 [  1   0   0   4  13   0   1  16   0   1   0   4]
 [  0   0   0   1   1   7   0   0  32   1  17   0]
 [  0   0   0   9   0   2   0   0   1  33   0   3]
 [  0   0   0   3   1   6   2   0  12   0  31   0]
 [  1   0   0   2   3   4   0   1   0  18   0  22]]
              precision    recall  f1-score   support

           1       0.95      0.96      0.95       580
           2       0.96      0.92      0.94       568
           3       0.98      0.96      0.97       537
           4       0.79      0.86      0.83       668
           5       0.83     

Model 7
0.8963743263106321
[[553  18   2   0   3   0   1   0   0   0   0   3]
 [ 32 529   2   0   2   0   0   1   0   0   1   1]
 [  0  31 504   0   0   0   0   0   0   0   0   2]
 [  0   0   0 576  87   0   2   1   0   2   0   0]
 [  0   0   0  70 628   0   1   3   1   1   4   0]
 [  0   0   0   1   0 714   0   0   2   1   0   1]
 [  0   0   0   6  10   0  20   1   0   1  11   0]
 [  1   0   0   4   3   0   3  28   0   1   0   0]
 [  0   0   0   6   1   7   0   0  28   1  16   0]
 [  0   0   0   9   1   3   2   0   0  28   0   5]
 [  0   0   0   3   2   6   0   0  18   1  25   0]
 [  1   0   0   3   1   4   0   1   0  15   0  26]]
              precision    recall  f1-score   support

           1       0.94      0.95      0.95       580
           2       0.92      0.93      0.92       568
           3       0.99      0.94      0.96       537
           4       0.85      0.86      0.86       668
           5       0.85      0.89      0.87       708
           6       0.97      0.99  

Model 13
0.904703576678099
[[566   5   3   0   0   0   0   1   0   0   2   3]
 [ 37 503  26   0   1   0   0   1   0   0   0   0]
 [  2   5 529   0   0   0   0   0   0   0   1   0]
 [  0   0   0 621  44   0   1   0   1   1   0   0]
 [  0   0   0  91 608   1   2   1   0   0   4   1]
 [  0   0   0   0   0 716   0   0   1   0   0   2]
 [  0   0   0  10  11   0  23   2   1   0   2   0]
 [  0   0   0   4  13   0   0  22   0   0   0   1]
 [  0   0   0   1   3   7   0   0  44   0   4   0]
 [  0   0   0   8   0   3   0   0   0  32   0   5]
 [  0   0   0   4   1   5   1   1  30   0  13   0]
 [  1   0   0   2   4   4   0   1   0  23   0  16]]
              precision    recall  f1-score   support

           1       0.93      0.98      0.95       580
           2       0.98      0.89      0.93       568
           3       0.95      0.99      0.97       537
           4       0.84      0.93      0.88       668
           5       0.89      0.86      0.87       708
           6       0.97      1.00  